In [1]:
import sys

sys.path.insert(0, '..')
import os
from biosimulator_processes.process_bigraph import pp, pf
from biosimulator_processes.data_model import *
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder
from biosimulator_processes.builder.builder_api import test_data_model 
from biosimulator_processes.processes.copasi_process import CopasiProcess


model_filepath = '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'

os.path.exists(model_filepath)

CobraProcess registered successfully.
CopasiProcess registered successfully.
TelluriumProcess registered successfully.
DeterministicTimeCourseParameterScan registered successfully.


True

In [2]:
copasi_process_model = TimeCourseModel(
    model_id='BIOMODEL',
    model_source=model_filepath
)

In [3]:
b = BiosimulatorBuilder()

In [5]:
builder = test_data_model(
    builder=b, 
    process_name='CopasiProcess', 
    process_class=CopasiProcess, 
    model="{'model': {'model_source': model_filepath}}",
    method='stochastic')

AttributeError: You must pass a model.

In [ ]:
b.register_process('CopasiProcess', CopasiProcess)

In [ ]:
copasi_model = b.get_pydantic_model('CopasiProcess')

In [ ]:
copasi_model.model_fields

In [ ]:
reg = b.core.process_registry

In [ ]:
c = reg.get_pydantic_model('CopasiProcess')

In [ ]:
c.model_fields